In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import json
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime
import glob
import re
import io
from scipy.stats import boxcox

**1. Overview and Basic Analysis**

**1.1 Overview the dataset**

In [ ]:
df = pd.read_csv('../input/us-accidents/US_Accidents_June20.csv')
print("The shape of data is:",(df.shape))
display(df.head(5))

Details about features(49) in the dataset:


Traffic Attributes (12):
* ID: This is a unique identifier of the accident record.
* Source: Indicates source of the accident report (i.e. the API which reported the accident.).
* TMC: A traffic accident may have a Traffic Message Channel (TMC) code which provides more detailed description of the event.
* Severity: Shows the severity of the accident, a number between 1 and 4, where 1 indicates the least impact on traffic (i.e., short delay as a result of the accident) and 4 indicates a significant impact on traffic (i.e., long delay).
* Start_Time: Shows start time of the accident in local time zone.
* End_Time: Shows end time of the accident in local time zone.
* Start_Lat: Shows latitude in GPS coordinate of the start point.
* Start_Lng: Shows longitude in GPS coordinate of the start point.
* End_Lat: Shows latitude in GPS coordinate of the end point.
* End_Lng: Shows longitude in GPS coordinate of the end point.
* Distance(mi): The length of the road extent affected by the accident.
* Description: Shows natural language description of the accident.


Address Attributes (9):
* Number: Shows the street number in address field.
* Street: Shows the street name in address field.
* Side: Shows the relative side of the street (Right/Left) in address field.
* City: Shows the city in address field.
* County: Shows the county in address field.
* State: Shows the state in address field.
* Zipcode: Shows the zipcode in address field.
* Country: Shows the country in address field.
* Timezone: Shows timezone based on the location of the accident (eastern, central, etc.).


Weather Attributes (11):
* Airport_Code: Denotes an airport-based weather station which is the closest one to location of the accident.
* Weather_Timestamp: Shows the time-stamp of weather observation record (in local time).
* Temperature(F): Shows the temperature (in Fahrenheit).
* Wind_Chill(F): Shows the wind chill (in Fahrenheit).
* Humidity(%): Shows the humidity (in percentage).
* Pressure(in): Shows the air pressure (in inches).
* Visibility(mi): Shows visibility (in miles).
* Wind_Direction: Shows wind direction.
* Wind_Speed(mph): Shows wind speed (in miles per hour).
* Precipitation(in): Shows precipitation amount in inches, if there is any.
* Weather_Condition: Shows the weather condition (rain, snow, thunderstorm, fog, etc.).


POI Attributes (13):
* Amenity: A Point-Of-Interest (POI) annotation which indicates presence of amenity in a nearby location.
* Bump: A POI annotation which indicates presence of speed bump or hump in a nearby location.
* Crossing: A POI annotation which indicates presence of crossing in a nearby location.
* Give_Way: A POI annotation which indicates presence of give_way sign in a nearby location.
* Junction: A POI annotation which indicates presence of junction in a nearby location.
* No_Exit: A POI annotation which indicates presence of no_exit sign in a nearby location.
* Railway: A POI annotation which indicates presence of railway in a nearby location.
* Roundabout: A POI annotation which indicates presence of roundabout in a nearby location.
* Station: A POI annotation which indicates presence of station (bus, train, etc.) in a nearby location.
* Stop: A POI annotation which indicates presence of stop sign in a nearby location.
* Traffic_Calming: A POI annotation which indicates presence of traffic_calming means in a nearby location.
* Traffic_Signal: A POI annotation which indicates presence of traffic_signal in a nearby location.
* Turning_Loop: A POI annotation which indicates presence of turning_loop in a nearby location.


Period-of-Day (4):
* Sunrise_Sunset: Shows the period of day (i.e. day or night) based on sunrise/sunset.
* Civil_Twilight: Shows the period of day (i.e. day or night) based on civil twilight.
* Nautical_Twilight: Shows the period of day (i.e. day or night) based on nautical twilight.
* Astronomical_Twilight: Shows the period of day (i.e. day or night) based on astronomical twilight.

Lets separate the datasets based on data type so that we can make good analysis

In [ ]:
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df

In [ ]:
dtype_df.groupby("Column Type").aggregate('count').reset_index()

**1.2 Reporting Sources**

In [ ]:
df_source = df.groupby(['Severity','Source']).size().reset_index().pivot(\
    columns='Severity', index='Source', values=0)
df_source.plot(kind='bar', stacked=True, title='Severity Count by Sources')

In [ ]:
df.groupby('Severity').size()

The results indicate that there are two main sources, Bing and MapQuest. Meanwhile, we can see that most accidents are moderately severe in degree 2 and 3. 

**1.3 States**

****

In [ ]:
states = df.State.unique()

count_by_state=[]
for i in df.State.unique():
    count_by_state.append(df[df['State']==i].count()['ID'])

fig,ax = plt.subplots(figsize=(16,10))
sns.barplot(states,count_by_state)
plt.title("Count of Accidents by States", size=15, y=1.05)
plt.xlabel('States',fontsize=15)
plt.ylabel('Number of Accidents',fontsize=15)
plt.show()


California, Texas and Florida are the TOP 3 traffic accident prone states.

**1.4 Weather**

What weather conditions are most common when accidents occur?

In [ ]:
fig, ax=plt.subplots(figsize=(16,7))
df['Weather_Condition'].value_counts().sort_values(ascending=False).head(10).plot.bar(width=0.5,color='Teal',edgecolor='k',align='center',linewidth=2)
plt.xlabel('Weather Conditions',fontsize=20)
plt.ylabel('Number of Accidents',fontsize=20)
ax.tick_params(labelsize=20)
plt.title('Top 10 Weather Condition for accidents',fontsize=25)
plt.grid()
plt.ioff()

**1.5 Durations**

In [ ]:
start = pd.to_datetime(df.Start_Time, format='%Y-%m-%d %H:%M:%S')
end = pd.to_datetime(df.End_Time, format='%Y-%m-%d %H:%M:%S')
laps=end-start
top_15 = laps.astype('timedelta64[m]').value_counts().nlargest(15) 
#Return the first n rows ordered by columns in descending order.
print('Top 15 longest accidents correspond to {:.1f}% of the data'.format(top_15.sum()*100/len(laps)))
(top_15/top_15.sum()).plot.bar(figsize=(10,8), color = 'plum')
plt.title('Top 15 Accident Durations', fontsize = 24, color='indigo')
plt.xlabel('Duration in minutes')
plt.ylabel('% of Total Data')
plt.grid(linestyle=':', linewidth = '0.2', color ='salmon');

**1.6 Time**

In [ ]:
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df['End_Time'] = pd.to_datetime(df['End_Time'])
df['Month'] = df['Start_Time'].dt.month
df['Year'] = df['Start_Time'].dt.year
df['Hour'] = df['Start_Time'].dt.hour
df['Weekday'] = df['Start_Time'].dt.weekday
df['Duration'] = (df['End_Time'] - df['Start_Time']).dt.total_seconds()/60

# clean the data based on the condition that the impact on traffic is between zero-one week,
# and drop duplicates
oneweek = 60*24*7
df_clean = df[(df['Duration']>0) & (df['Duration']< oneweek)].drop_duplicates(subset=['Start_Time','End_Time','City','Street','Number','Description'])

In [ ]:
#time series analysis
df1 = df_clean[['Country','Start_Time','End_Time','Year','Month','Weekday','Hour','Duration','Severity']]


**Year**

Accidents are increasing with time.

In [ ]:
sns.set_style('whitegrid')
sns.set_context('talk')
sns.set_palette('GnBu_d')
a = sns.catplot(x='Year',data=df_clean[df_clean['Year'] < 2020],kind='count')
a.fig.suptitle('Yearly Accident Cases(2016-2019)',y=1.03)
a.set(ylabel='yearly cases',xlabel='year')
plt.show()

**Month**

There were more accident cases druing August and December compared to other months,excluding the data from 2020. It may be due to more bad weather conditions in the winter.


In [ ]:
sns.set_context('talk')
m = sns.catplot(x='Month',data=df1[df1['Year'] < 2019],kind='count')
m.fig.suptitle('Monthly Accident Cases(2016-2019)',y=1.03)
m.set(ylabel='monthly cases')
plt.show()


**Week**

It looks like that more accidents happened on working day from Mon to Fri than on weekend.

In [ ]:
sns.set_context('talk')
w = sns.catplot(x='Weekday',data=df1,kind='count')
w.fig.suptitle('Weekday Accident Cases',y=1.03)
w.set(ylabel='weekday cases')
plt.show()

**Hour**

It shows that the most accidents occur at around 7-8 am in the morning and 4 to 5 pm when most people travel between home and work. The increases of the traffic density could lead to more accidents.

In [ ]:
sns.set_context('paper')
h = sns.catplot(x='Hour',data=df1,kind='count')
h.fig.suptitle('Hourly accidents cases',y=1.03)
h.set(ylabel='hourly cases',xlabel='hour')
plt.annotate('morning peak',xy=(6,330000))
plt.annotate('afternoon peak',xy=(15,270000))
plt.annotate('bottom',xy=(1,25000))
plt.annotate('go to work',xy=(7.5,0),xytext=(1,125000),arrowprops={'arrowstyle':'fancy'})
plt.annotate('get off work',xy=(17.5,0),xytext=(19,150000),arrowprops={'arrowstyle':'fancy'})
plt.show()

**2. Data Processing and Cleaning**

**Plotting correlation matrix**

We can use this to remove highly correlated features like keeping only one of wind chill and temperature as they both mean almost the same thing.

In [ ]:
fig=plt.gcf()
fig.set_size_inches(20,20)
fig=sns.heatmap(df_clean.corr(),annot=True,linewidths=0.2,linecolor='k',square=True,mask=False, vmin=-1, vmax=1,cbar_kws={"orientation": "vertical"},cbar=True)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

**2.1 Drop meaningless features**

Features 'Source' and 'ID' couldn't provide any useful information about accidents themselves. In addition, 'Start_time' 'End_Time' (we have year, month, weekday, hour and duration), 'End_Lat', and 'End_Lng'(we save start location) can be collected only after the accident has already happened and hence cannot be predictors for serious accident prediction. For 'TMC' and 'Description', the POI features have already been extracted from it by dataset creators. Let's get rid of these features first.

In [ ]:
df_clean = df_clean.drop(['ID','TMC','Source','Description', 'End_Time', 'End_Lat', 'End_Lng', 'Distance(mi)','Duration'], axis=1)

Drop 'Country' and 'Turning_Loop' which have only one class.

In [ ]:
cat_names = ['Side', 'Country', 'Timezone', 'Amenity', 'Bump', 'Crossing', 
             'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 
             'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 
             'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
print("Unique count of categorical features:")
for i in cat_names:
  print(i,df[i].unique().size)

In [ ]:
df_clean = df_clean.drop(['Country','Turning_Loop'], axis=1)
df_clean.columns

**2.2 Handling missing data**

2.2.1 Get the ratio and the columns with missing values

In [ ]:
missing_df = df_clean.isnull().sum(axis=0).reset_index()
missing_df.columns = ['columns_name','missing_count']
missing_df['missing_ratio'] = missing_df['missing_count'] /df_clean.shape[0]
missing_df.loc[missing_df['missing_ratio']>0]

2.2.2 Missing more than 50%

'Number' and 'Wind_Chill(F)' will be dropped because they are not highly related to severity according to previous research, whereas 'Precipitation(in)' could be a useful predictor and hence can be handled by separating feature.Drop 'Number', 'Wind_Chill(F)'. 
Add a new feature for missing values in 'Precipitation(in)' and replace missing values with median.

In [ ]:
missin = missing_df.loc[missing_df['missing_count']>500000]
removelist = missin['columns_name'].tolist()
removelist

In [ ]:
df_clean = df_clean.drop(['Number','Wind_Chill(F)'], axis=1)

In [ ]:
df_clean['Precipitation_NA'] = 0
df_clean.loc[df_clean['Precipitation(in)'].isnull(),'Precipitation_NA'] = 1
df_clean['Precipitation(in)'] = df_clean['Precipitation(in)'].fillna(df_clean['Precipitation(in)'].median())
df_clean.loc[:5,['Precipitation(in)','Precipitation_NA']]

2.2.3 Missing less than 1% 

Drop NaN by features:
There still are some missing values but much less. Just dropna by these features for the sake of simplicity.


In [ ]:
df_clean = df_clean.dropna(subset=['City','Zipcode','Timezone','Airport_Code',
                       'Sunrise_Sunset','Civil_Twilight',
                       'Nautical_Twilight','Astronomical_Twilight'])

2.2.4 Others

The other absent values will be imputed by employing the most common value
for categorical features and the median for continuous ones.

Weather Data with missing values

> - Continuous weather features with missing values:
Temperature(F),
Humidity(%),
Pressure(in),
Visibility(mi),
Wind_Speed(mph),

Before imputation, weather features will be grouped by location and time first, to which weather is naturally related. 'Airport_Code' is selected as location feature because the sources of weather data are airport-based weather stations. Then the data will be grouped by 'Start_Month' rather than 'Start_Hour' because using the former is computationally cheaper and remains less missing values. Finally, missing values will be replaced by median value of each group.

In [ ]:
# group data by 'Airport_Code' and 'Start_Month' then fill NAs with median value
Weather_data=['Temperature(F)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Speed(mph)']
print("The number of remaining missing values: ")
for i in Weather_data:
  df_clean[i] = df_clean.groupby(['Airport_Code','Month'])[i].apply(lambda x: x.fillna(x.median()))
  print( i + " : " + df_clean[i].isnull().sum().astype(str))


In [ ]:
df_clean = df_clean.dropna(subset=Weather_data)

- Categorical Weather Features:

For categorical weather features, majority rather than median will be used to replace missing values.

In [ ]:
# group data by 'Airport_Code' and 'Start_Month' then fill NAs with majority value
from collections import Counter
weather_cat = ['Wind_Direction','Weather_Condition']
print("Count of missing values that will be dropped: ")
for i in weather_cat:
  df_clean[i] = df_clean.groupby(['Airport_Code','Month'])[i].apply(lambda x: x.fillna(Counter(x).most_common()[0][0]) if all(x.isnull())==False else x)
  print(i + " : " + df_clean[i].isnull().sum().astype(str))

In [ ]:
# drop na
df_clean = df_clean.dropna(subset=weather_cat)

**2.3 Transformation and Simplification of Categorical features**

For categorical variables, some attributes have more than 20
levels, leading to numerous attributes after dummy coding. Therefore, it would be more practical to simplify
them into a few levels. Moreover, relative old features with raw data, outliers and negative features can be removed.

2.3.1 Time features

Time features has been normalized during EDA overview chapter. 




2.3.2 Weather features

- Wind Direction

In [ ]:
print("Wind Direction: ", df_clean['Wind_Direction'].unique())

In [ ]:
#Simplify wind direction
df = df_clean
df.loc[df['Wind_Direction']=='Calm','Wind_Direction'] = 'CALM'
df.loc[(df['Wind_Direction']=='West')|(df['Wind_Direction']=='WSW')|(df['Wind_Direction']=='WNW'),'Wind_Direction'] = 'W'
df.loc[(df['Wind_Direction']=='South')|(df['Wind_Direction']=='SSW')|(df['Wind_Direction']=='SSE'),'Wind_Direction'] = 'S'
df.loc[(df['Wind_Direction']=='North')|(df['Wind_Direction']=='NNW')|(df['Wind_Direction']=='NNE'),'Wind_Direction'] = 'N'
df.loc[(df['Wind_Direction']=='East')|(df['Wind_Direction']=='ESE')|(df['Wind_Direction']=='ENE'),'Wind_Direction'] = 'E'
df.loc[df['Wind_Direction']=='Variable','Wind_Direction'] = 'VAR'
print("Wind Direction after simplification: ", df['Wind_Direction'].unique())

- Weather Condition

Weather-related vehicle accidents kill more people annually than large-scale weather disasters(source: weather.com). According to Road Weather Management Program, wet-pavement during rainfall, winter-condition and fog are the main reasons for weather-related accidents. To extract these three weather conditions, we first look at what we have in 'Weather_Condition' Feature.

In [ ]:
# show distinctive weather conditions 
weather ='!'.join(df['Weather_Condition'].dropna().unique().tolist())
weather = np.unique(np.array(re.split(
    "!|\s/\s|\sand\s|\swith\s|Partly\s|Mostly\s|Blowing\s|Freezing\s", weather))).tolist()
print("Weather Conditions: ", weather)

In [ ]:
#Create features for some common weather conditions.
df['Clear'] = np.where(df['Weather_Condition'].str.contains('Clear', case=False, na = False), 1, 0)
df['Cloud'] = np.where(df['Weather_Condition'].str.contains('Cloud|Overcast', case=False, na = False), 1, 0)
df['Rain'] = np.where(df['Weather_Condition'].str.contains('Rain|storm', case=False, na = False), 1, 0)
df['Heavy_Rain'] = np.where(df['Weather_Condition'].str.contains('Heavy Rain|Rain Shower|Heavy T-Storm|Heavy Thunderstorms', case=False, na = False), 1, 0)
df['Snow'] = np.where(df['Weather_Condition'].str.contains('Snow|Sleet|Ice', case=False, na = False), 1, 0)
df['Heavy_Snow'] = np.where(df['Weather_Condition'].str.contains('Heavy Snow|Heavy Sleet|Heavy Ice Pellets|Snow Showers|Squalls', case=False, na = False), 1, 0)
df['Fog'] = np.where(df['Weather_Condition'].str.contains('Fog', case=False, na = False), 1, 0)


Drop old features with raw data

In [ ]:
df = df.drop(['Start_Time','Weather_Timestamp','Weather_Condition'], axis=1)

- Continuous Weather Features

Normalize features with extreamly skewed distribution.

In [ ]:
df['Pressure_bc']= boxcox(df['Pressure(in)'].apply(lambda x: x+1),lmbda=6)
df['Visibility_bc']= boxcox(df['Visibility(mi)'].apply(lambda x: x+1),lmbda = 0.1)
df['Wind_Speed_bc']= boxcox(df['Wind_Speed(mph)'].apply(lambda x: x+1),lmbda=-0.2)
df = df.drop(['Pressure(in)','Visibility(mi)','Wind_Speed(mph)'], axis=1)

In [ ]:
#Change Name Back
df['Pressure(in)'] = df['Pressure_bc']
df['Visibility(mi)'] = df['Visibility_bc']
df['Wind_Speed(mph)'] = df['Wind_Speed_bc']
df = df.drop(['Pressure_bc','Visibility_bc','Wind_Speed_bc'], axis=1)


In [ ]:
df.Year = df.Year.astype(str)
sns.countplot(x='Year', hue='Severity', data=df ,palette="Set2")
plt.title('Count of Accidents by Year', size=15, y=1.05)
plt.show()

**3. Feature Engineering**

3.1 Encoding Features

3.1.1 Address features

- Frequency Encoding

Some location features that have too many unique values can be labeled by their frequency. Frequency encoding and log-transform:

'Street'

'City'

'County'
 
'Zipcode'

'Airport_Code'

In [ ]:
fre_list = ['Street', 'City', 'County', 'Zipcode', 'Airport_Code']
for i in fre_list:
  newname = i + '_Freq'
  df[newname] = df.groupby([i])[i].transform('count')
  df[newname] = df[newname]/df.shape[0]*df[i].unique().size
  df[newname] = df[newname].apply(lambda x: np.log(x+1))

In [ ]:
df = df.drop(fre_list, axis  = 1)

3.1.2 POI features

In [ ]:
POI_features = ['Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal']

fig, axs = plt.subplots(ncols=3, nrows=4, figsize=(15, 10))

plt.subplots_adjust(hspace=0.5,wspace = 0.5)
for i, feature in enumerate(POI_features, 1):    
    plt.subplot(3, 4, i)
    sns.countplot(x=feature, hue='Severity', data=df, palette="Set2")
    
    plt.xlabel('{}'.format(feature), size=12, labelpad=3)
    plt.ylabel('Accident Count', size=12, labelpad=3)    
    plt.tick_params(axis='x', labelsize=12)
    plt.tick_params(axis='y', labelsize=12)
    
    plt.legend(['1','2','3','4'], loc='upper right', prop={'size': 10})
    plt.title('Count of Severity in {}'.format(feature), size=14, y=1.05)
fig.suptitle('Count of Accidents in POI Features',y=1.02, fontsize=16)
plt.show()

Accidents near traffic signal and crossing are much less likely to be serious accidents while little more likely to be serious if they are near the junction. Maybe it is because people usually slow down in front of crossing and traffic signal but junction and severity are highly related to speed. Other POI features are so unbalanced that it is hard to tell their relation with severity from plots.

In [ ]:
df= df.drop(['Bump','Give_Way','No_Exit','Roundabout','Traffic_Calming'], axis=1)

3.2 Correlation

In [ ]:
#drop features with duplicated meanings and high correlation
df = df.drop(['Civil_Twilight', 'Nautical_Twilight', 
              'Astronomical_Twilight'], axis=1)


In [ ]:
df['Severity'] = df['Severity'].astype(int)
plt.figure(figsize=(20,20))
cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)
sns.heatmap(df.corr(), annot=True,cmap=cmap, center=0).set_title("Correlation Heatmap", fontsize=16)
plt.show()

In [ ]:
#drop high corrolated features
df = df.drop(['County_Freq','Zipcode_Freq',
              'Airport_Code_Freq','Year'], axis=1)

In [ ]:
df = df.replace([True, False], [1,0])

In [ ]:
df.to_csv('/kaggle/working/df_lessF.csv')

In [ ]:
df.info()

In [ ]:
#Check if it contains Null value again.
missingdf = df.isnull().sum(axis=0).reset_index()
missingdf.columns = ['columns_name','missing_count']
missingdf['missing_ratio'] = missingdf['missing_count'] /df.shape[0]
missingdf.loc[missingdf['missing_ratio']>0]

3.3 Data Resampling

In [ ]:
df["Severity"].value_counts()

In [ ]:
#df_bl = df.sample(frac=0.05) 
#not as good as resampling. Directly resampling!

In [ ]:
df_bl = pd.concat([df[df['Severity']==2].sample(250000, random_state=42),
                   df[df['Severity']==3].sample(25000, random_state=42),
                   df[df['Severity']==4].sample(500000, replace = True, random_state=42),
                   df[df['Severity']==1].sample(7500, replace = True, random_state=42)], axis=0)

df_bl["Severity"].value_counts()

In [ ]:
df_bl = df_bl.drop(['Street_Freq','City_Freq'],axis=1)

3.4 One-hot Encoding

In [ ]:
# Generate dummies for categorical data
cat = ['Side','State','Timezone','Wind_Direction', 'Weekday', 'Month', 'Hour','Sunrise_Sunset']
df_bl[cat] = df_bl[cat].astype('category')
df_bl = pd.get_dummies(df_bl, columns=cat, drop_first=True)
df_bl.head()

In [ ]:
df_bl.info()

4. Model Building

In [ ]:
# Import DecisionTreeClassifier from sklearn.tree
from sklearn.tree import DecisionTreeClassifier

# Import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

# Import LogisticRegression
from sklearn.linear_model import LogisticRegression

#Import Others
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc

In [ ]:
# List of classification algorithms
algo_lst=['Logistic Regression','Decision Trees','Random Forest', 
          'Support Vector Machine', 'Neural network', 'XGBoost' ]

# Initialize an empty list for the accuracy score for each algorithm
accuracy_lst=[]

# Initialize an empty list for the f1 score for each algorithm
f1_lst=[]

In [ ]:
X = df_bl.drop('Severity',axis=1)
y = df_bl['Severity']


# split train test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(\
              X, y, test_size=0.20, random_state=42, stratify=y)

In [ ]:
#----------------Logistic regression
from sklearn.linear_model import LogisticRegression
import time
tic2 = time.time()
lr = LogisticRegression(max_iter=1000,random_state=42)
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)

# Get the accuracy score
acc=accuracy_score(y_test, y_pred)

# Get the f1 score
f1_lr=f1_score(y_test, y_pred, average='weighted')

# Append to the accuracy list
accuracy_lst.append(acc)
f1_lst.append(f1_lr)

# Display results
print("[Logistic regression algorithm] accuracy_score: {:.3f}.".format(acc))
print("[Logistic regression algorithm] f1_score: {:.3f}.".format(f1_lr))
toc2 = time.time()
print('Elapsed time for Losigtic regression is %f seconds \n' % float(toc2 - tic2))

In [ ]:
#------------------Decision tree algorithm
# 1. Instantiate dt_entropy, set 'entropy' as the information criterion
dt_entropy = DecisionTreeClassifier(criterion='entropy')

# 1. Fit dt_entropy to the training set
dt_entropy.fit(X_train, y_train)

# 1. Use dt_entropy to predict test set labels
y_pred= dt_entropy.predict(X_test)

# 1. Evaluate accuracy_entropy
accuracy_entropy = accuracy_score(y_test, y_pred)

# 1. Get the f1 score
f1_dt_entropy=f1_score(y_test, y_pred, average='weighted')

# Append to the accuracy list
#accuracy_lst.append(acc)
#f1_lst.append(f1_lr)

# 1. Print accuracy_entropy
print('[Decision Tree -- entropy] accuracy_score: {:.3f}.'.format(accuracy_entropy))
print("[Decision Tree -- entropy] f1_score: {:.3f}.".format(f1_dt_entropy))

tic3 = time.time()

# 2. Instantiate dt_gini, set 'gini' as the information criterion
dt_gini = DecisionTreeClassifier(criterion='gini')

# 2. Fit dt_entropy to the training set
dt_gini.fit(X_train, y_train)

# 2. Use dt_entropy to predict test set labels
y_pred= dt_gini.predict(X_test)

# 2. Evaluate accuracy_entropy
accuracy_gini = accuracy_score(y_test, y_pred)

# 2. Get the f1 score
f1_dt_gini=f1_score(y_test, y_pred, average='weighted')

# 2. Append to the accuracy list
acc=accuracy_gini
accuracy_lst.append(acc)
f1_lst.append(f1_dt_gini)

# 2. Print accuracy_gini
print('[Decision Tree -- gini] accuracy_score: {:.3f}.'.format(accuracy_gini))
print("[Decision Tree -- gini] f1_score: {:.3f}.".format(f1_dt_gini))

toc3 = time.time()
print('Elapsed time for Decision tree is %f seconds \n' % float(toc3 - tic3))


In [ ]:
# --------------Random Forest algorithm
tic4 = time.time()
#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=150)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

# Get the accuracy score
acc=accuracy_score(y_test, y_pred)

# Get the f1 score
f1_clf=f1_score(y_test, y_pred, average='weighted')

# Append to the accuracy list
accuracy_lst.append(acc)
f1_lst.append(f1_clf)

# Model Accuracy, how often is the classifier correct?
print("[Randon forest algorithm] accuracy_score: {:.3f}.".format(acc))
print("[Randon forest algorithm] f1_score: {:.3f}.".format(f1_clf))
toc4 = time.time()
print('Elapsed time for Randon forest is %f seconds \n' % float(toc4 - tic4))

In [ ]:
import pickle
filename = '80wRF.sav'
#pickle.dump(clf, open(filename, 'wb'))
pickle.dump(clf, open('80wRF.pickle', 'wb'))

In [ ]:
#Algorithm Random Forest
#Visualize important features
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_rows', 350)
pd.set_option('display.max_columns', 350)
plt.style.use('ggplot')

feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)

# Creating a bar plot, displaying only the top k features
k=20
sns.barplot(x=feature_imp[:20], y=feature_imp.index[:k])
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

In [ ]:
# List top k important features
k=20
feature_imp.sort_values(ascending=False)[:k]

In [ ]:
#Algorithm Random Forest
#Select the top important features, set the threshold
# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0.03
sfm = SelectFromModel(clf, threshold=0.03)

# Train the selector
sfm.fit(X_train, y_train)

feat_labels=X.columns

# Print the names of the most important features
for feature_list_index in sfm.get_support(indices=True):
    print(feat_labels[feature_list_index])

In [ ]:
# Transform the data to create a new dataset containing only the most important features
# Note: We have to apply the transform to both the training X and test X data.
X_important_train = sfm.transform(X_train)
X_important_test = sfm.transform(X_test)

# Create a new random forest classifier for the most important features
clf_important = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)

# Train the new classifier on the new dataset containing the most important features
clf_important.fit(X_important_train, y_train)

In [ ]:
# Apply The Full Featured Classifier To The Test Data
y_pred = clf.predict(X_test)

# View The Accuracy Of Our Full Feature Model
print('[Randon forest algorithm -- Full feature] accuracy_score: {:.3f}.'.format(accuracy_score(y_test, y_pred)))

# Apply The Full Featured Classifier To The Test Data
y_important_pred = clf_important.predict(X_important_test)

# View The Accuracy Of Our Limited Feature Model
print('[Randon forest algorithm -- Limited feature] accuracy_score: {:.3f}.'.format(accuracy_score(y_test, y_important_pred)))

#View The F1 Score of Our Full Feature Model
print('[Randon forest algorithm -- Full feature] f1_score: {:.3f}.'.format(f1_score(y_test, y_pred, average='weighted')))

# View The F1 Score Of Our Limited Feature Model
print('[Randon forest algorithm -- Limited feature] f1_score: {:.3f}.'.format(f1_score(y_test, y_important_pred, average='weighted')))

In [ ]:
import pickle
filename = 'RF_limit.sav'
#pickle.dump(clf, open(filename, 'wb'))
pickle.dump(clf_important, open('RF_limit.pickle', 'wb'))

In [ ]:
#--------------- Support vector machine algorithm
from sklearn import svm
tic5 = time.time()

svm = svm.SVC(gamma='auto')
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

# Get the accuracy score
acc=accuracy_score(y_test, y_pred)

# Get f1-score
f1_svm=f1_score(y_test, y_pred, average='weighted')

# Append to the accuracy list
accuracy_lst.append(acc)
f1_lst.append(f1_svm)

print("[Support vector machine algorithm] accuracy_score: {:.3f}.".format(acc))
print("[Support vector machine algorithm] f1_score: {:.3f}.".format(f1_svm))
toc5 = time.time()
print('Elapsed time for Support vector machine is %f seconds \n' % float(toc5 - tic5))

In [ ]:
#--------------- Neural network algorithm
from sklearn import neural_network
import time

tic6 = time.time()
nn = neural_network.MLPClassifier(hidden_layer_sizes=(5, 2))
nn.fit(X_train,y_train)
y_pred=nn.predict(X_test)

# Get the accuracy score
acc=accuracy_score(y_test, y_pred)

# f1_score
f1_nn=f1_score(y_test, y_pred, average='weighted')

# Append to the accuracy list
accuracy_lst.append(acc)
f1_lst.append(f1_nn)

print("[Neural network algorithm] accuracy_score: {:.3f}.".format(acc))
print("[Neural network algorithm] f1_score: {:.3f}.".format(f1_nn))
toc6 = time.time()
print('Elapsed time for Neural network is %f seconds \n' % float(toc6 - tic6))

In [ ]:
#--------------- XGBoost algorithm
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
tic7 = time.time()
xgb = XGBClassifier(objective= 'multi:softmax',num_class=4,n_fold=4,
                    colsample_bytree = 1,
                    learning_rate = 0.15,
                    max_depth = 5,
                    n_estimators = 600,
                    subsample = 0.3)
xgb.fit(X_train, y_train)

y_pred=xgb.predict(X_test)

# Get the accuracy score
acc=accuracy_score(y_test, y_pred)

# Get f1 score
f1_xgb=f1_score(y_test, y_pred, average='weighted')

# Append to the accuracy list
accuracy_lst.append(acc)
f1_lst.append(f1_xgb)


print("[XGBoost algorithm] accuracy_score: {:.3f}.".format(acc))
print("[XGBoost algorithm] f1_score: {:.3f}.".format(f1_xgb))
toc7 = time.time()
print('Elapsed time for Losigtic regression is %f seconds \n' % float(toc7 - tic7))

In [ ]:
# Make a plot of the accuracy scores for different algorithms

# Generate a list of ticks for y-axis
y_ticks=np.arange(len(algo_lst))

# Combine the list of algorithms and list of accuracy scores into a dataframe, sort the value based on accuracy score
df_acc=pd.DataFrame(list(zip(algo_lst, accuracy_lst)), columns=['Algorithm','Accuracy_Score']).sort_values(by=['Accuracy_Score'],ascending = True)

# Export to a file
df_acc.to_csv('./Accuracy_scores_algorithms.csv',index=False)

# Make a plot
ax=df_acc.plot.barh('Algorithm', 'Accuracy_Score', align='center',legend=False,color='0.5')

# Add the data label on to the plot
for i in ax.patches:
    # get_width pulls left or right; get_y pushes up or down
    ax.text(i.get_width()+0.02, i.get_y()+0.2, str(round(i.get_width(),2)), fontsize=10)


# Set the limit, lables, ticks and title
plt.xlim(0,1.1)
plt.xlabel('Accuracy Score')
plt.yticks(y_ticks, df_acc['Algorithm'], rotation=0)
plt.title('Which algorithm is better regarding accuracy score?')

plt.show()

In [ ]:
# Make a plot of the f1 scores for different algorithms

# Generate a list of ticks for y-axis
y_ticks=np.arange(len(algo_lst))

# Combine the list of algorithms and list of accuracy scores into a dataframe, sort the value based on accuracy score
df_acc=pd.DataFrame(list(zip(algo_lst, f1_lst)), columns=['Algorithm','F1_Score']).sort_values(by=['F1_Score'],ascending = True)

# Export to a file
df_acc.to_csv('./f1_scores_algorithms.csv',index=False)

# Make a plot
ax=df_acc.plot.barh('Algorithm', 'F1_Score', align='center',legend=False,color='0.5')

# Add the data label on to the plot
for i in ax.patches:
    # get_width pulls left or right; get_y pushes up or down
    ax.text(i.get_width()+0.02, i.get_y()+0.2, str(round(i.get_width(),2)), fontsize=10)


# Set the limit, lables, ticks and title
plt.xlim(0,1.1)
plt.xlabel('F1 Score')
plt.yticks(y_ticks, df_acc['Algorithm'], rotation=0)
plt.title('Which algorithm is better regarding f1 score?')

plt.show()